In [ ]:
from imutils import paths 
from shutil import copyfile
import pandas as pd
import numpy as np
import cv2
import csv
import os

In [ ]:
def make_dir(dirName):
    # Create a target directory & all intermediate 
    # directories if they don't exists
    if not os.path.exists(dirName):
        os.makedirs(dirName, exist_ok = True)
        print("[INFO] Directory " , dirName ,  " created")
    else:
        print("[INFO] Directory " , dirName ,  " already exists")

In [ ]:
SOURCE_PATH = 'TFW/'
TARGET_PATH = 'dataset/YOLOv5'

TRAIN_IMG_DIR = 'train/images'
TRAIN_LAB_DIR = 'train/labels'
make_dir(os.path.join(TARGET_PATH, TRAIN_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, TRAIN_LAB_DIR))

TEST_IMG_DIR = 'test/images'
TEST_LAB_DIR = 'test/labels'
make_dir(os.path.join(TARGET_PATH, TEST_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, TEST_LAB_DIR))

VAL_IMG_DIR = 'val/images'
VAL_LAB_DIR = 'val/labels'
make_dir(os.path.join(TARGET_PATH, VAL_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, VAL_LAB_DIR))

In [ ]:
def convert(source_path, target_path, set_='train', env='indoor'):
    img_path = os.path.join(source_path, set_, env+"_gray")
    csv_file = os.path.join(source_path, set_, '{}.csv'.format(env))
    
    # import labels as a numpy array
    labels = pd.read_csv(csv_file,header=0).to_numpy()

    # initialize iterators
    iter1 = 0 
    iter2 = 0

    # init face counter
    face_count = 0

    # loop over the rows
    while iter1 < len(labels):
        # load the image
        imageName = labels[iter1][0]
        
        source_img = os.path.join(img_path, imageName)
        image = cv2.imread(source_img)
        H, W = image.shape[:2]
        
        target_img = os.path.join(target_path, set_, 'images', imageName)
        target_txt = open(os.path.join(target_path, set_,
                                'labels/{}.txt'.format(imageName.split('.png')[0])), "w")
        
        copyfile(source_img, target_img)
        
        # loop over the labels of this image
        while labels[iter1][0] == labels[iter2][0]:
            # extract coordinates of the bounding box and five facial landmarks
            xs, ys, xe, ye, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, p5x, p5y = labels[iter2][1:]
            label = 'face {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n'.format(xs, ys, xe-xs, ye-ys, p1x, p1y, p2x, p2y, 
                                                                         p3x, p3y, p4x, p4y, p5x, p5y)
            w = xe - xs
            h = ye - ys
                
            xc = xs + w / 2
            yc = ys + h / 2
                
            label = '0 {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f}\n'.format(
                    xc/W, yc/H, w/W, h/H, p1x/W, p1y/H, p2x/W, p2y/H, p3x/W, p3y/H, p4x/W, p4y/H, p5x/W, p5y/H)
                    
            target_txt.write(label)
            face_count += 1
                
            iter2 += 1
                
            # break the loop if we exceeded
            # the last row
            if iter2 == len(labels):
                break
        
        # equalize iterators
        iter1 = iter2

    print(iter2, face_count)

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='train', env='indoor')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='train', env='outdoor')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='val', env='indoor')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='val', env='outdoor')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='test', env='indoor')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='test', env='outdoor')